**LIVE CCTV**

The project aims at recording reasonable changes detected in a video and also recognising the object that was responsible for the change in the video.

The libraries used in the project are:
* [OpenCV](https://anaconda.org/conda-forge/opencv)
* [numpy](https://anaconda.org/anaconda/numpy)

In [ ]:
#Importing important libraries
import cv2
import numpy as np

**EUCLIDEAN DISTANCE**

The project uses euclidean distance as a measure of difference between the frames of the video

Euclidean distance is given by (sigma[(x-y)^2])^0.5

There are other methods of differentiating between 2 images, functions for some like:
* Cosine Similarity
* Manhattan Distance

These are given in other the folder named _Other Methods_


In [ ]:
def euclidean_distance(image1, image2):
    img1_array = np.asarray(image1)  #converts old_frame to an array
    img2_array = np.asarray(image2)  #converts frame into an array
#     print(image2)
    
    gray_img1_array = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray_img2_array = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    
    #Formula for Euclidean distance
    #euclidean_distance = (sigma[(x-y)^2])^0.5
    eucdis = np.linalg.norm(gray_img1_array - gray_img2_array)
    return eucdis

**VIDEO CAPTURING AND CONTOUR DETECTION**

* The below code accesses the webcam of your device and captures video using cv2.
* This code further detects Contours and assigns bounding boxes to the moving objects detected in the image by the webcam.
* Euclidean distance function is used to store only the frames of the video in which the movemenr is detected.
* Also feel free to use other methods of image differetation given the the _Other Methods_ folder.
  * Remember to change the threshold of video recording accordingly
* The program also prints out the euclidean distance everytime it reaches ahead of the threshold

In [ ]:
cap = cv2.VideoCapture(0)                      #opens the webcam
ret = True                                     #creates a boolean 
ret, old_frame = cap.read()                    #ret is true and the first frame of video saved in old_frame

#subrtacting background from the objects
back_sub = cv2.createBackgroundSubtractorMOG2(history=700, 
           varThreshold=25, detectShadows=True)

#Height and wdith of the frames in which video will be written
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
   
size = (frame_width, frame_height)
   
# Below VideoWriter will create object
# a frame of above defined The output 
# is stored in 'Recording.avi' file.
rec_vid = cv2.VideoWriter('Recording.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         5, size)

kernel = np.ones((30,30),np.uint8) 
# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while ret:
    ret, frame = cap.read()          #saves the first frame of video in frame 
    
    #uses the euclidean function created above to calculate the similarit
    sime = euclidean_distance(old_frame, frame) 
    
    #applies the background subtractor
    fg_mask = back_sub.apply(frame)
    
    #applies CLOSE MORPHOLOGY to remove black spots (removes noises)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    
    #blurring filter for lesser noise and distortion
    fg_mask = cv2.medianBlur(fg_mask, 5)
    
    #converts RGB to binary image assigns 0 to pixels less than treshold and 255 to more than threshold
    _, fg_mask = cv2.threshold(fg_mask,127,255,cv2.THRESH_BINARY)
    
    fg_mask_bb = fg_mask
    
    #detects Contours i.e Detects the change of colour in the image
    contours, hierarchy = cv2.findContours(fg_mask_bb,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(c) for c in contours] #creates arrays of areas detected in different contours
    
    #if no contour detected
    if len(areas) < 1:
        cv2.imshow('frame',frame)
        continue
    #if contour detected
    else:
        max_index = np.argmax(areas) #take the largest movement causing object(area)
        
    cnt = contours[max_index] #Takes contours of the maximum area
    x,y,w,h = cv2.boundingRect(cnt) #Takes the center co-ordinates , height and width
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3) #prints out a bounding box in the image
    
    if(sime>94000):                  #Threshold set to 94000 for euclidean distance 
        print(sime)                  #Prints the euclidean distance
        rec_vid.write(frame)         #Records video when movement detected
    
    cv2.imshow('Input', frame)   #opens the webcam in a pop-up window
    old_frame = frame            #saves the vale of the new frame in old frame to be used later in the loop
    c = cv2.waitKey(1)           #new frame comes after () ms
    if cv2.waitKey(1) & 0xFF == ord('q'): #press q on keyboard to stop the webcam
        break
    
    
cap.release()
cv2.destroyAllWindows()          #Once out of the while loop, the pop-up window closes automatically